Import tools

In [2]:
import pandas as pd
from nltk.stem import PorterStemmer, WordNetLemmatizer
from rank_bm25 import BM25L
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
import pickle
import os
from operator import itemgetter
from collections import Counter
nltk.download('stopwords')
stemmer = PorterStemmer()

FLAG_TOKENIZE = False

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Antoine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Import data

In [3]:
# Import corpus and queries as a dataframe
corpus = pd.read_json('Data/corpus.jsonl', lines=True)
queries = pd.read_json('Data/queries.jsonl', lines=True)

In [76]:
test_corpus_id = [1396701, 1396704, 1396705, 1396707, 1396708, 1453630, 1605506, 
                  1652605, 1772930, 1772932, 1815965, 1907207, 1972028, 2025485, 
                  2077310, 210379, 210384, 2210591, 2210592, 2210593, 2210595, 
                  2210597, 2232789, 2393704, 2401465, 252142, 2588143, 2627023, 
                  275724, 275726, 275728, 275732, 2872589, 2923493, 2923494, 2923496, 
                  2923497, 2923498, 2923501, 2969050, 3143377, 3203819, 3305804, 3357360, 
                  3385968, 3385971, 3450898, 3485337, 3523599, 3523602, 3538160, 3538161, 
                  3538162, 3538163, 3538164, 3538165, 3538166, 3538167, 3538168, 3545107, 
                  3573471, 3573474, 3573477, 3573478, 3573479, 3918680, 3930148, 4044822, 
                  4245927, 4257001, 4272446, 4279298, 4279299, 4289598, 4378740, 4393631, 
                  4549698, 4566815, 4566816, 4566818, 4566819, 4566822, 4566823, 4656722, 
                  4691129, 4801563, 4834224, 4834226, 4889894, 4910037, 4927817, 4998398, 
                  5007631, 5063813, 5117497, 5359304, 5379124, 5568031, 5652031, 5702414, 
                  5774013, 5774014, 5866302, 5931262, 5931263, 5931264, 5931265, 5931266, 
                  5931267, 5931268, 5931269, 5931270, 5931271, 6142152, 6181638, 6226401, 
                  6260059, 6263225, 6351110, 6462880, 6521993, 6761344, 6761347, 6923052, 
                  6923054, 6923055, 6923056, 6944106, 7082922, 7176430, 7313730, 7313733, 
                  7327683, 7565849, 7643828, 7709320, 7742098, 7816726, 7832420, 7837084, 
                  7837086, 7861747, 7861748, 7861749, 7861750, 7861751, 7861753, 7861755, 
                  7861756, 7885529, 8002846, 8002850, 8002851, 8002852, 8002853, 8002854, 
                  8002855, 8029524, 8029527, 8029530, 8029531, 8029533, 8103063, 8103065, 
                  8103066, 8103067, 8103068, 8178996, 8178997, 8178998, 8179000, 8179001, 
                  8179003, 82105, 82106, 82107, 82108, 82109, 82110, 82111, 82113, 82114, 
                  8304084, 8402970, 8402971, 8402972, 8420020, 8455300, 8455301, 850454, 94879, 98515]

test_corpus_ranks = [0, 0, 1, 0, 2, 0, 3, 0, 0, 0, 2, 1, 2, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 2, 3, 2, 0, 3, 2, 2, 2, 3, 2, 0, 0, 0, 0, 1, 3, 2, 0, 1, 0, 0, 3, 2, 0, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 3, 0, 0, 0, 2, 0, 1, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 2, 2, 0, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 1, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 1, 0, 2, 2, 0, 0, 2, 2, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 0, 3, 2, 2, 1, 2, 3, 1, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 0, 3, 0, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0]


test_corpus_text = corpus[corpus['_id'].isin(test_corpus_id)]

test_query_id = 915593

test_query_text = queries[queries['_id']==test_query_id]

In [77]:
nltk.download('punkt')  # Download the Punkt tokenizer data if you haven't already
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')  # Download the stopwords list if you haven't already

stop_words = set(stopwords.words('english'))

test_documents = [word_tokenize(document) for document in test_corpus_text['text']]
test_documents = [list(filter(lambda x: x.lower() not in stop_words and x.isalpha() and x.isalpha(), doc)) for doc in test_documents]
test_documents = list(map(lambda x : list(map(lambda y : stemmer.stem(y.lower()), x)), test_documents))
bm25 = BM25L(corpus=test_documents)
query = [word_tokenize(document) for document in test_query_text['text']]
query = [list(filter(lambda x: x.lower() not in stop_words and x.isalpha(), doc)) for doc in query]
query = list(map(lambda x : list(map(lambda y : stemmer.stem(y.lower()), x)), query))[0]
scores = bm25.get_scores(query)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mathiaskroismoller/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mathiaskroismoller/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mathiaskroismoller/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
sorted_corpus_ids = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
sorted_corpus_ids = [(test_corpus_id[x[0]], test_corpus_ranks[x[0]]) for x in sorted_corpus_ids]
print(sorted_corpus_ids)

[(465438, 3), (622472, 0), (4314173, 0), (64045, 0), (6150628, 0), (6816993, 2), (5232955, 0), (4314172, 3), (8108818, 2), (3025463, 3), (331549, 0), (5483597, 3), (7625204, 0), (5605160, 0), (2471196, 0), (2785222, 3), (4487937, 0), (2164297, 0), (235241, 0), (848507, 3), (4675180, 0), (5888683, 1), (600966, 0), (7615797, 0), (7210883, 0), (580734, 0), (7007569, 2), (8065457, 0), (3227499, 0), (414967, 0), (3081626, 2), (4606188, 0), (8188169, 2), (4314171, 2), (4902727, 3), (5257694, 0), (5264931, 0), (7129173, 2), (655776, 3), (1470412, 0), (3222646, 0), (5605159, 1), (5790943, 0), (8495256, 2), (612896, 3), (1593008, 0), (638673, 0), (5900625, 3), (2785223, 2), (4051141, 3), (6989198, 2), (6653839, 2), (6492656, 0), (5900622, 0), (660826, 2), (4487939, 0), (480791, 0), (5014104, 0), (3469357, 2), (2382279, 0), (2230480, 0), (5599692, 0), (3907355, 0), (5747664, 0), (580736, 2), (3203280, 3), (1540837, 2), (8106320, 0), (8106322, 2), (4314174, 0), (1732923, 2), (4314169, 0), (810631

# Compute complete list of queries and .csv output

In [79]:
# Read the task2_test.tsv file
test_data = pd.read_csv('/Users/mathiaskroismoller/DistribInfoSystems/Project1/Task2/Data/task2_test.tsv', delimiter='\t')

# Function to preprocess and rank documents for a given query
def rank_documents(query, documents):
    # Tokenize, filter stopwords, and stem the query
    query_tokens = [stemmer.stem(token.lower()) for token in word_tokenize(query) if token.lower() not in stop_words and token.isalpha()]

    # Tokenize, filter stopwords, and stem the documents
    doc_tokens = [[stemmer.stem(token.lower()) for token in word_tokenize(doc) if token.lower() not in stop_words and token.isalpha()] for doc in documents]

    bm25 = BM25L(corpus=doc_tokens)
    scores = bm25.get_scores(query_tokens)
    
    return scores

results = []

### Ranking function

In [80]:
# Process each query and rerank documents
for index, row in test_data.iterrows():
    query_id = row['id']  
    corpus_ids = row['corpus-id']
    query_text = queries[queries['_id'] == row['query-id']]['text'].values[0]
    corpus_ids = [int(id_str.strip()) for id_str in corpus_ids.strip('[]').split(',')]
    documents = corpus[corpus['_id'].isin(corpus_ids)]['text'].tolist()
    
    # Rank documents for the query
    scores = rank_documents(query_text, documents)
    
    # Store results in the desired format
    score_str = '[' + ', '.join(map(str, scores)) + ']' 
    result = {
        'id': query_id,
        'corpus-id': -1,
        'score': score_str
    }
    results.append(result)

result_df = pd.DataFrame(results)
result_df.head()

,id,corpus-id,score
0,7437,-1,"[1.1217166209269143, 1.0122838799703053, 1.012..."
1,7438,-1,"[9.093759939593262, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
2,7439,-1,"[8.391079526234456, 8.016550911790427, 1.21074..."
3,7440,-1,"[0.0, 1.0029426437798896, 0.0, 2.6048273068387..."
4,7441,-1,"[7.186386273410823, 5.069382411282289, 2.91554..."


### Output .csv file

In [67]:
# Write the results to a CSV file
result_df.to_csv('output.csv', index=False)